# This is a **cleared** version of notebook used during data preparation. Something may not work!

In [1]:
import pickle
import bs4

from multiprocessing import Pool
import istarmap
import tqdm
from parser import *

In [2]:
d = {}

for i in range(0, 18000, 1000):
    with open(f'from_{i}_to_{i+999}.pickle', 'rb') as f:
        d.update(pickle.load(f))

In [ ]:
# validate all links
# check html page by searching for h1 title

def validate():
    with open('key-title_list.txt', 'w', encoding='utf-8') as t:
        for key in d.keys():
            try:
                t.write(f'{key:7}' + bs4.BeautifulSoup(d[key]).find('h1').text.strip() + '\n')
            except AttributeError as e:
                print(key)

In [27]:
validate()

4073
1428
111123
6346
102390
7862
145278
2192
8724
8023
8423
18391
104348
98142
20891
121065
138685
138912
1821


In [4]:
invalid_links = ['/anime/' + i for i in '''
    4073
    1428
    111123
    6346
    102390
    7862
    145278
    2192
    8724
    8023
    8423
    18391
    104348
    98142
    20891
    121065
    138685
    138912
    1821
'''.split()]

invalid_links_2 = ['/anime/' + i for i in '''926
129591
4477
115435
18549
131574'''.split()]

In [5]:
driver = Parser.init_driver()

In [111]:
invalid_data = Parser.parse_links(driver, invalid_links)

100%|████████████████████████████████████████████████████████████████████████████████████████████████| 19/19 [00:47<00:00,  2.52s/it]


In [6]:
invalid_data_2 = Parser.parse_links(driver, invalid_links_2)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:14<00:00,  2.45s/it]


In [121]:
with open('invalid_data_fix.pickle', 'wb') as f:
    pickle.dump(invalid_data, f, protocol=pickle.HIGHEST_PROTOCOL)

In [7]:
with open('invalid_data_fix_2.pickle', 'wb') as f:
    pickle.dump(invalid_data_2, f, protocol=pickle.HIGHEST_PROTOCOL)

In [3]:
with open('invalid_data_fix.pickle', 'rb') as f:
    invalid_data = pickle.load(f)

In [4]:
with open('invalid_data_fix_2.pickle', 'rb') as f:
    invalid_data_2 = pickle.load(f)

In [5]:
d.update(invalid_data)

In [6]:
d.update(invalid_data_2)

In [7]:
tasks = []

for key in d.keys():
    tasks.append((d[key], key))

In [8]:
if __name__ == "__main__":
    with Pool(16) as pool:
        data = list(tqdm.tqdm(pool.istarmap(extract_data_task, tasks), total=len(tasks), miniters=1))

100%|█████████████████████████████████████████████████████████████████████████████████████████| 17250/17250 [01:06<00:00, 258.29it/s]


In [26]:
data_dict = {}

for i in data:
    if bool(i):
        data_dict[list(i.keys())[0]] = list(i.values())[0]

In [12]:
import pandas as pd

In [56]:
df = pd.DataFrame.from_dict(data_dict, orient='index').reset_index()
df.to_csv('anilist_data.csv', index=False)